## Crawl Job 104

In [2]:
!pip install pymysql

In [12]:
import requests
from bs4 import BeautifulSoup
import pymysql
import time

In [15]:
resp = requests.get("http://www.104.com.tw/i/apis/jobsearch.cfm?cat=2007001004&fmt=8&cols=JOB")
pages = int(resp.json()["TOTALPAGE"]) + 1
url_template = "http://www.104.com.tw/i/apis/jobsearch.cfm?cat=2007001004&fmt=8&cols=JOB,NAME,PCSKILL_ALL_DESC,ADDR_NO_DESCRIPT,ADDRESS&PAGES={}"

conn = pymysql.connect(host='localhost', port=3306, user='root', password='1234', db='jobs', use_unicode=True, charset="utf8", autocommit=True)
cur = conn.cursor()
insert_104_template = "insert into Job(site, company, job, addr, skill) values(104, %s, %s, %s, %s)"

for page in range(1, pages):
    time.sleep(1) # 每秒爬一次避免還未resp回來程式就往下跑
    print('page: ' + str(page))
    url = url_template.format(page)
    json = requests.get(url).json()
    if 'data' in json:
        datas = json['data']
    else:
        continue

    # 若沒該欄位則給空值
    for data in datas:
        if 'PCSKILL_ALL_DESC' not in data:
            data['PCSKILL_ALL_DESC'] = ''
        if 'ADDR_NO_DESCRIPT' not in data:
            data['ADDR_NO_DESCRIPT'] = ''
        if 'ADDRESS' not in data:
            data['ADDRESS'] = ''

    jobs = [(data['NAME'], data['JOB'], data['ADDR_NO_DESCRIPT']+data['ADDRESS'], data['PCSKILL_ALL_DESC']) for data in datas]
    cur.executemany(insert_104_template, jobs)
    
cur.close
conn.close
# cur.execute("SELECT * FROM job")
# for row in cur:
#     print(row)

    

page: 1
page: 2
page: 3
page: 4
page: 5
page: 6
page: 7
page: 8
page: 9
page: 10
page: 11
page: 12
page: 13
page: 14
page: 15
page: 16
page: 17
page: 18
page: 19
page: 20
page: 21
page: 22
page: 23
page: 24
page: 25
page: 26
page: 27
page: 28
page: 29
page: 30
page: 31
page: 32
page: 33
page: 34
page: 35
page: 36
page: 37
page: 38
page: 39
page: 40
page: 41
page: 42
page: 43
page: 44
page: 45
page: 46
page: 47
page: 48
page: 49
page: 50
page: 51
page: 52
page: 53
page: 54
page: 55
page: 56
page: 57
page: 58
page: 59
page: 60
page: 61
page: 62
page: 63
page: 64
page: 65
page: 66
page: 67
page: 68
page: 69
page: 70
page: 71
page: 72
page: 73
page: 74
page: 75
page: 76
page: 77
page: 78
page: 79
page: 80
page: 81
page: 82
page: 83
page: 84
page: 85
page: 86
page: 87
page: 88
page: 89
page: 90
page: 91
page: 92
page: 93
page: 94
page: 95
page: 96
page: 97
page: 98
page: 99
page: 100
page: 101
page: 102
page: 103
page: 104
page: 105
page: 106
page: 107
page: 108
page: 109
page: 110
page: 11

<bound method Connection.close of <pymysql.connections.Connection object at 0x02B40E50>>

## Count Languages

In [81]:
cur.execute('select skill from jobs.job')
skills_dict = {}
languages_1 = ['SQL', 'HTML', 'C#', 'PHP', 'JAVASCRIPT', 'LINUX', 'C++', 'JQUERY'] # 可模糊比對語言 Ex. 無論TSQL或OL/SQL只會被計算為一次SQL
languages_2 = ['C', 'JAVA'] # 精準比對語言 Ex. Java 不會在 Javascript 被算入

for row in cur:
    skills = str.split(row[0], sep=' ') # DB資料室以空格區分技術
    skills = [str.upper(skill) for skill in skills]
    
    if len(skills) == 1 and skills[0] == '': 
        continue # 若該工作沒有技術要求，則跳過到下筆工作
    else:
        languages_used = set() # 用Set來確保一種語言只有一份
        
         # 模糊比對語言
        for item in languages_1:
            for skill in skills:
                if item in skill:
                    languages_used.add(item)
                    
        # 精準比對
        for item in [language for language in languages_2 if language in skills]:
            languages_used.add(item)
        
        if languages_2 in skills:
            languages_used.add('C')
    
    # 建立出現次數
    for language in languages_used:
        if language not in skills_dict:
            skills_dict[language] = 1
        else:
            skills_dict[language] = int(skills_dict[language]) + 1

skills_dict

{'C': 2701,
 'C#': 4338,
 'C++': 2325,
 'HTML': 3120,
 'JAVA': 2423,
 'JAVASCRIPT': 3706,
 'JQUERY': 3547,
 'LINUX': 638,
 'PHP': 912,
 'SQL': 4456}